In [1]:
import osmium
from osmium.geom import haversine_distance
import os
import math
from scipy.spatial import KDTree
import numpy as np
import random

out_file = "../osm/cost.osm"
input_file = '../osm/divided.osm'

In [2]:
def check_cost_node(n: osmium.Node):
    x = 0.5
    tag = n.tags.get('highway', '')
    if tag == 'crossing':
        x += 20
    elif tag == 'traffic_signals':
        x += 35
    return x

In [3]:
def check_cost(w: osmium.Way, node_costs: dict[int, float]):
    length = haversine_distance(w.nodes)
    lanes:float
    if w.tags.get("highway", "") in ("construction", "service"):
        lanes = 3.
    else:
        lanes = float(w.tags.get('lanes', '2'))
        if not (w.tags.get("oneway", "") == "yes" or w.tags.get("junction", "") == "roundabout" \
                or w.tags.get("junction", "") == "circular" and not w.tags.get("oneway", "") == "no"):
            lanes /= 2

    surface_mul: float
    surface =  w.tags.get('surface', '')
    if not surface:
        surface_mul = 1.3
    elif surface in ['asphalt']:
        surface_mul = 1
    elif surface in ['paved', 'concrete']:
        surface_mul = 1.1
    elif surface in ['concrete:plates', 'metal', 'paving_stones']:
        surface_mul = 1.4
    elif surface in ['compacted']:
        surface_mul = 1.6
    elif surface in ['fine_gravel']:
        surface_mul = 1.9
    elif surface in ['unpaved', 'gravel']:
        surface_mul = 2.1
    elif surface in ['ground', 'dirt']:
        surface_mul = 2.5
    else:
        print(surface)
        surface_mul = 1.5

    cost = length
    if lanes < 3:
        cost += cost * (3 - lanes) * 0.5
    cost *= surface_mul
    return cost

In [4]:
class OSMHandler(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.ways = []
        self.nodes = []
        self.relations = []
        self.node_costs: dict[int, float] = {}
        
    def node(self, n: osmium.Node):
        self.nodes.append((n.id, n.location, dict(n.tags)))
        self.node_costs[n.id] = check_cost_node(n)

    def way(self, w: osmium.Way):
        if 'highway' in w.tags:
            cost = check_cost(w, self.node_costs)
            for i in w.nodes:
                cost += self.node_costs[i.ref]
            cost += 20
            self.ways.append((w.id, dict(w.tags), cost, list(w.nodes)))
        else:
            print("Input files expected to contain only highways")

    def relation(self, r: osmium.Relation):
        self.relations.append((r.id, dict(r.tags), list(r.members)))

In [5]:
def save_modified_ways(ways, nodes_way, relations, writer):

    for id, loc, tags in nodes_way:
        node = osmium.osm.mutable.Node()
        node.id = id
        node.location = loc 
        node.tags = tags
        writer.add_node(node)
        
    for way_id, tags, cost, nods in ways:
        new_way = osmium.osm.mutable.Way()
        new_way.id = way_id
        new_way.tags = tags
        new_way.tags['cost'] = f"{cost:.2f}"
        new_way.nodes = nods
        writer.add_way(new_way)

    for id, tags, members in relations:
        new_rel = osmium.osm.mutable.Relation()
        new_rel.id = id
        new_rel.tags = tags
        new_rel.members = members
        writer.add_relation(new_rel)
    writer.close()

In [6]:
if os.path.exists(out_file):
    os.remove(out_file)

writer = osmium.SimpleWriter(out_file)

try:
    h = OSMHandler()
    h.apply_file(input_file, locations=True)
    save_modified_ways(h.ways, h.nodes, h.relations, writer)
finally:
    writer.close()  